# AHBA C3 Projection Analysis (Interactive)

This notebook performs the AHBA gene weight projection on the downsampled dataset (`combined_postnatal_full_harmony_10k.h5ad`).
It replicates the logic from `scripts/project_genes.py` and uses `rpy2` for plotting.

In [94]:
import scanpy as sc
import pandas as pd
import numpy as np
import sys
import os
import re
import warnings
%load_ext autoreload
%autoreload 2

# Suppress warnings
warnings.filterwarnings('ignore')

# Load R extension at the top
%load_ext rpy2.ipython

# Define Base Directory for Portability
base_dir = "/home/rajd2/rds/"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
%%R

library(ggplot2)
library(dplyr)
library(readr)
library(patchwork)
library(ggpubr)
library(stringr)

In [91]:
# Add code directory to path for regulons module
# This assumes the notebook is running inside the 'notebooks' folder of the repo
sys.path.append(os.path.join(base_dir, "hpc-work/snRNAseq_2026/code/"))
try:
    from regulons import get_ahba_GRN, project_GRN
    from metadata_utils import get_original_metadata
except ImportError as e:
    print(f"Error importing modules: {e}")

## Helper Functions

In [ ]:
def map_ensembl_to_symbol_from_ref(adata, ref_path):
    """
    Maps Ensembl IDs (in adata.var_names) to Gene Symbols using a reference H5AD file.
    """
    print(f"Loading reference mapping from {ref_path}...")
    try:
        # Load only var from reference to save memory
        ref_adata = sc.read_h5ad(ref_path, backed='r')
        # Assumption: ref_adata.var_names are Ensembl IDs, ref_adata.var['gene_name'] are Symbols
        if 'gene_name' not in ref_adata.var.columns:
             print("Error: 'gene_name' column not found in reference .var")
             return adata
             
        # Create dictionary
        ensembl_to_symbol = ref_adata.var['gene_name'].to_dict()
        print(f"Loaded {len(ensembl_to_symbol)} gene mappings.")
        
        # Apply mapping
        new_names = [ensembl_to_symbol.get(idx, idx) for idx in adata.var_names]
        
        adata.var['gene_symbol'] = new_names
        adata.var_names = new_names
        adata.var_names_make_unique()
        print("Gene mapping complete. var_names updated to symbols (unique).")
        
    except Exception as e:
        print(f"Mapping failed: {e}")
        
    return adata

def extract_age_years(age_str):
    age_str = str(age_str).lower()
    # Years
    match = re.search(r"(\d+)\s?-?year", age_str)
    if match:
        return float(match.group(1))
    
    # Months
    match = re.search(r"(\d+)\s?-?month", age_str)
    if match:
        return float(match.group(1)) / 12.0
        
    return np.nan

def assign_age_range(age):
    if np.isnan(age):
        return "Unknown"
    if age <= 2: return "Infancy"
    if age <= 12: return "Childhood"
    if age <= 20: return "Adolescence"
    return "Adulthood"

def abbreviate_cell_type(ctype):
    ctype = str(ctype)
    if "intratelencephalic" in ctype:
        return ctype.replace("intratelencephalic projecting glutamatergic neuron", "IT").replace("intratelencephalic projecting glutamatergic cortical neuron", "IT")
    if "extratelencephalic" in ctype:
        return ctype.replace("extratelencephalic projecting glutamatergic cortical neuron", "ET").replace("extratelencephalic projecting glutamatergic neuron", "ET")
    if "corticothalamic" in ctype:
        return ctype.replace("corticothalamic-projecting glutamatergic cortical neuron", "CT").replace("corticothalamic-projecting glutamatergic neuron", "CT")
    if "near-projecting" in ctype:
        return ctype.replace("near-projecting glutamatergic neuron", "NP").replace("near-projecting glutamatergic cortical neuron", "NP")
    if "L6b" in ctype:
        return "L6b"
    return ctype

## Load Data

In [ ]:
# Define absolute paths using base_dir
input_file = os.path.join(base_dir, "rds-cam-psych-transc-Pb9UGUlrwWc/Cam_snRNAseq/combined/combined_postnatal_full_harmony_10k.h5ad")
ref_mapping_file = os.path.join(base_dir, "rds-cam-psych-transc-Pb9UGUlrwWc/Cam_PsychAD/RNAseq/HBCC_Cohort_10k.h5ad")
grn_file = os.path.join(base_dir, "hpc-work/snRNAseq_2026/reference/ahba_dme_hcp_top8kgenes_weights.csv")

print(f"Loading data from {input_file}...")
adata = sc.read_h5ad(input_file)

In [133]:
# Standardize metadata from original sources
print("Standardizing metadata from original sources...")

# Map Ensembl to Symbols using REFERENCE (already in your notebook)
adata = map_ensembl_to_symbol_from_ref(adata, ref_mapping_file)

# Extract all standardized metadata (simplified cell types, regions, age, sex, donor)
orig_meta = get_original_metadata(adata.obs, base_dir)
for col in orig_meta.columns:
    adata.obs[col] = orig_meta[col]

# Derived fields for plotting
if 'age_years' in adata.obs.columns:
    adata.obs['Age_Years'] = adata.obs['age_years']
    adata.obs['Age_log2'] = np.log2(adata.obs['Age_Years'] + 1)
    adata.obs['Age_Range4'] = adata.obs['Age_Years'].apply(assign_age_range)
if 'donor_id' in adata.obs.columns:
    adata.obs['Individual'] = adata.obs['donor_id']
    
print("Metadata standardization complete.")
display(adata.obs[['cell_class', 'cell_type', 'region', 'dataset_source']].head())

Standardizing metadata from original sources...
Loading reference mapping from /home/rajd2/rds/rds-cam-psych-transc-Pb9UGUlrwWc/Cam_PsychAD/RNAseq/HBCC_Cohort_10k.h5ad...
Loaded 34176 gene mappings.
Gene mapping complete. var_names updated to symbols (unique).
Extracting original metadata...
Loading metadata from sources: ['HBCC' 'AGING' 'VELMESHEV' 'WANG']
Processing VELMESHEV (21563 cells)...
Processing WANG (10425 cells)...
Processing HBCC (34548 cells)...
  HBCC: Loaded 34548 exact matches, 0 stripped matches.
  HBCC processed successfully.
Processing AGING (26784 cells)...
  AGING: Loaded 26784 exact matches, 0 stripped matches.
  AGING processed successfully.
Metadata standardization complete.


,cell_class,cell_type,region,dataset_source
Donor_494-1-AAACCCACACAAACGG-0,Excitatory,L6 CT,prefrontal cortex,HBCC
Donor_494-1-AAAGGTATCGGAAACG-0,Excitatory,L2/3 IT,prefrontal cortex,HBCC
Donor_494-1-AACACACCAGTCGGAA-0,Excitatory,L2/3 IT,prefrontal cortex,HBCC
Donor_494-1-AAGCATCCAGCGTATT-0,Excitatory,L2/3 IT,prefrontal cortex,HBCC
Donor_494-1-AAGTACCCAAGAAACT-0,Excitatory,L5/6 NP,prefrontal cortex,HBCC


In [135]:
adata.obs[['dataset_source','cell_type']].value_counts().unstack(0)

dataset_source,AGING,HBCC,Velmeshev,Wang
cell_type,,,,
Excitatory,NaN,NaN,14782.0,2656.0
IT,NaN,NaN,1.0,NaN
L2/3 IT,22307.0,28606.0,3389.0,2655.0
L4 IT,NaN,NaN,3008.0,4073.0
L5 ET,44.0,39.0,NaN,NaN
L5 IT,NaN,NaN,383.0,NaN
L5/6 NP,714.0,1050.0,NaN,123.0
L6 CT,2788.0,3654.0,NaN,683.0
L6b,931.0,1199.0,NaN,235.0


In [136]:
adata.obs[['dataset_source','region']].value_counts().unstack(0)

dataset_source,AGING,HBCC,Velmeshev,Wang
region,,,,
cingulate cortex,NaN,NaN,3940.0,NaN
motor cortex,NaN,NaN,214.0,NaN
neocortex,NaN,NaN,717.0,NaN
prefrontal cortex,26784.0,34548.0,14663.0,5008.0
temporal cortex,NaN,NaN,2029.0,NaN
visual cortex,NaN,NaN,NaN,5417.0


In [113]:
# Restore Original Cell Types (Detailed)
if 'cell_class' not in adata.obs.columns:
    print("Standardizing metadata from original files...")
    datasets_to_load = ['VELMESHEV', 'WANG', 'AGING', 'HBCC']
    orig_meta = get_original_metadata(adata.obs, base_dir, datasets_to_load)
    
    for col in orig_meta.columns:
        adata.obs[col] = orig_meta[col]
            
    # Overwrite Cell_Type with cell_class if available for better plotting?
    # User logic: "align variables... for cell_class, cell_subclass, and cell_type"
    pd.set_option('display.max_columns', None)
    display(adata.obs[['cell_class', 'cell_subclass']].head())

In [137]:
# Filter for Excitatory Neurons (Optional, to mimic job)
filter_column = 'region' # Use the standardized column if available
filter_value = 'prefrontal cortex'

if filter_column in adata.obs.columns:
    print(f"Filtering for {filter_column} == {filter_value}...")
    n_start = adata.n_obs
    adata = adata[adata.obs[filter_column] == filter_value].copy()
    print(f"Filtered: {n_start} -> {adata.n_obs} cells kept.")

Filtering for region == prefrontal cortex...
Filtered: 93320 -> 81003 cells kept.


## Projection

In [138]:
print(f"Loading GRN from {grn_file}...")
ahba_GRN = get_ahba_GRN(path_to_ahba_weights=grn_file)

print("Projecting GRN...")
project_GRN(adata, ahba_GRN, 'X_ahba', use_highly_variable=True, log_transform=False)
print(f"Projected shape: {adata.obsm['X_ahba'].shape}")

Loading GRN from /home/rajd2/rds/hpc-work/snRNAseq_2026/reference/ahba_dme_hcp_top8kgenes_weights.csv...
Projecting GRN...
Found 4307 matching genes in var_names.
Loading expression data for 663 matched genes...
Projected shape: (81003, 6)


## Prepare Data for R Plotting

In [142]:
# Extract projection
projection_df = pd.DataFrame(adata.obsm['X_ahba'], index=adata.obs_names, columns=adata.uns['X_ahba_names'])
projection_df['obs_names'] = projection_df.index

melted = projection_df.melt(id_vars=['obs_names'], var_name='C', value_name='value')

# Merge with metadata
cols_to_keep = ['Individual', 'Cell_Type', 'Age_log2', 'Age_Years', 'Age_Range4', 'cell_class', 'cell_subclass', 'cell_type', 'source']
cols_to_keep = [c for c in cols_to_keep if c in adata.obs.columns]

meta = adata.obs[cols_to_keep].copy()
meta['obs_names'] = meta.index

final_df = pd.merge(melted, meta, on='obs_names')
final_df = final_df[final_df['C'].isin(['C3+', 'C3-'])]

# Display head
final_df.head()

,obs_names,C,value,Individual,Cell_Type,Age_log2,Age_Years,Age_Range4,cell_class,cell_subclass,cell_type,source
4,Donor_494-1-AAACCCACACAAACGG-0,C3+,21997.403587,Donor_494,Excitatory,4.754888,26.0,Adulthood,Excitatory,EN_L6_IT_2,L6 CT,HBCC
5,Donor_494-1-AAACCCACACAAACGG-0,C3-,8619.254898,Donor_494,Excitatory,4.754888,26.0,Adulthood,Excitatory,EN_L6_IT_2,L6 CT,HBCC
10,Donor_494-1-AAAGGTATCGGAAACG-0,C3+,44465.588135,Donor_494,Excitatory,4.754888,26.0,Adulthood,Excitatory,EN_L2_3_IT,L2/3 IT,HBCC
11,Donor_494-1-AAAGGTATCGGAAACG-0,C3-,3805.899128,Donor_494,Excitatory,4.754888,26.0,Adulthood,Excitatory,EN_L2_3_IT,L2/3 IT,HBCC
16,Donor_494-1-AACACACCAGTCGGAA-0,C3+,23699.008423,Donor_494,Excitatory,4.754888,26.0,Adulthood,Excitatory,EN_L3_5_IT_1,L2/3 IT,HBCC


## Plotting with R (rpy2)

In [ ]:
%%R -i final_df -i base_dir -w 220 -h 120 -u mm -r 300

# Source the plotting functions from code directory
source_path <- file.path(base_dir, "hpc-work/snRNAseq_2026/code/age_plots.r")
message(paste("Sourcing functions from", source_path))
source(source_path)

message("Processing data in R...")

# Clean/Process Data for R
df <- final_df %>%
  mutate(age_range = factor(Age_Range4, levels=c("Infancy", "Childhood", "Adolescence", "Adulthood"))) %>%
  filter(!is.na(age_range)) %>%
  mutate(C = factor(C, levels=c('C3+', 'C3-')))

# Plot 1: Age Scatter (Colored by 'cell_class' now that it's available)
message("Generating Plot 1: Age Scatter...")

color_col <- "source"
message(paste("Coloring by:", color_col))

p_age <- df %>% plot_age(color_var = color_col)

# Plot 2: Boxplots
message("Generating Plot 2: Boxplots...")
comparisons <- list(
    c('Adolescence', 'Adulthood'),
    c('Adolescence', 'Childhood')
)

df_box <- df %>% rename(network = C)
p_boxes <- df_box %>% plot_boxes(color_var='source') + stat_compare_means(comparisons = comparisons, color='blue', label='p.signif')

# Combine
p_final <- (p_age | p_boxes) + 
    plot_annotation(tag_levels='a', title="AHBA C3 in 4 datasets")

# Display
p_final

Error in `group_by()`:
! Must group by variables found in `.data`.
✖ Column `dataset_source` is not found.
Run `rlang::last_trace()` to see where the error occurred.
Sourcing functions from /home/rajd2/rds//hpc-work/snRNAseq_2026/code/age_plots.r
Processing data in R...
Generating Plot 1: Age Scatter...
Coloring by: source
Generating Plot 2: Boxplots...
Error in group_by(., network, Individual, age_range, !!sym(color_var)) : 
✖ Column `dataset_source` is not found.


RInterpreterError: Failed to parse and evaluate line '\n# Source the plotting functions from code directory\nsource_path <- file.path(base_dir, "hpc-work/snRNAseq_2026/code/age_plots.r")\nmessage(paste("Sourcing functions from", source_path))\nsource(source_path)\n\nmessage("Processing data in R...")\n\n# Clean/Process Data for R\ndf <- final_df %>%\n  mutate(age_range = factor(Age_Range4, levels=c("Infancy", "Childhood", "Adolescence", "Adulthood"))) %>%\n  filter(!is.na(age_range)) %>%\n  mutate(C = factor(C, levels=c(\'C3+\', \'C3-\')))\n\n# Plot 1: Age Scatter (Colored by \'cell_class\' now that it\'s available)\nmessage("Generating Plot 1: Age Scatter...")\n\ncolor_col <- "source"\nmessage(paste("Coloring by:", color_col))\n\np_age <- df %>% plot_age(color_var = color_col)\n\n# Plot 2: Boxplots\nmessage("Generating Plot 2: Boxplots...")\ncomparisons <- list(\n    c(\'Adolescence\', \'Adulthood\'),\n    c(\'Adolescence\', \'Childhood\')\n)\n\ndf_box <- df %>% rename(network = C)\np